# Bandit Problem 

## 什么是多臂赌博机问题

> 假设赌博机有$n$个手臂，玩家$Player_A$每摇一次手臂需要花费一枚代币，当玩家$Player_A$花费一枚代币任意摇一次手臂$a$后，赌博机会随机吐出金币的数量为$r_t(a)$，玩家获得这些金币后继续进行下一次的摇手臂动作。这些金币$r_{t}(a)$表示玩家$Player_A$在进行第$t$次动作时去摇手臂$a$获得的回报，摇一次手臂$a$叫做执行一次动作$A_t=a$，因为赌博机只有$n$个手臂，因此可执行的动作是有限的$n$个动作$A_t=a,a=1,2,...,n$。如果玩家总共只有$T$个代币，那么玩家最多只有$T$次摇手臂的机会，在$T$次动作执行完之后，把所有回报相加就得到累积回报$R_T = \sum_{t=1}^T r_{t}$。那么很自然地引出多臂赌博机的__基本问题：如何在有限次时间长度$T$的时间序列上去分布这些动作$A_t(a)$，才能获得最大的累积回报$R_T$？__     
> 为了回答该问题，本文首先对基本问题作进一步地数学解释，然后给出任何算法都不能超越的上下界，并介绍当前的一些算法以及这些算法之间的区别，最后给出一个好的赌博机算法应当具备品质，以及如何寻求新的算法。    

## 回报、价值函数与目标函数

> **回报函数$r_t(A_t)$**: 回报函数是摇手臂实验的一个随机变量。因为如果把摇手臂看作是一次实验，在第$t$次动作选择中，选择摇手臂$a$的动作记为$A_t=a$，并获得回报为$r_t(A_t=a)$，这个回报即为该次实验结果对应的一个数，实验结果与数的对应规则就被回报函数所规定的。从赌博机外表来看，每个动作执行之后赌博机将会回到开始状态，另一个动作继续从这个开始状态开始，由于我们不知道赌博机内部的细节，也无法确定依次执行的两个动作之间是否对内部的状态有影响，假设这里没有影响。      

> **累积回报函数$R_T$**: 累积回报是从开始摇手臂到当前这个时间段里获得的所有金币数量的总和，即$R_T = \sum_{t=1}^T r_{t}$，对于某个动作$a$来说，可以定义该动作的累积回报$R_T(a)$。
$$R_T = \sum_{t=1}^T r_{t}$$   
$$R_T(a) = \sum_{t=1}^T r_{t}(A_t=a)$$    
但是，因为每次获得的奖励是服从一个随机分布的，因此累积回报的具体数值没有意义，因此需关注累积回报的期望值，即 
$$R_T = E[\sum_{t=1}^T r_{t}]$$    
$$R_T(a) = E[\sum_{t=1}^T r_{t}(A_t=a)]$$   
    
> **动作的价值函数$q^*(a)$与$Q_t(a)$及样本平均方法**: 在第$i$次选择时，选择的动作为$A_i=a$，并获得回报$r_i(A_i=a)$，因为每次执行动作$a$都会获得不同的回报，因此对于动作$a$在每次执行时会存在一个期望回报，这个期望回报称为该动作的价值$q^*(a)$。由于我们只能观察获得回报，因此不能提前预先知道该动作的价值是多少，但是经过$t-1$次选择后，我们可以根据观察结果去估计价值$Q_t(a)$，根据大数定理，只要观察的次数足够多，可以得到该动作的平均回报，用平均回报作为估计的价值$Q_t(a)$，便可以使估计价值$Q_t(a)$非常接近真实价值$q^*(a)$。 对于所有动作来说，最大价值对应的动作为最优动作$A^*$，最大的估计价值对应的动作为当前最优估计动作$A_t^*$。
$$q^*(a)=E[r_t|A_t = a]$$   
$$Q_t(a) = \frac{1}{N_a}*R_t(a) = \frac{1}{N_a}*\sum_{i=1}^{t-1} r_{i}(A_i=a) ，N_a为在时间步t时前t-1次选择中动作a被选择的次数$$   
$$A^* = arg\max_{a} q^*(a) $$  
$$A_t^* = arg\max_{a}Q_t(a) $$ 
$$q^* = arg\max_{a}q^*(a) $$ 

> **动作$a$的估计价值函数$Q_t(a)$的样本平均方法的增量式实现**:    
$$Q_{n+1} = \frac{1}{n}*\sum_{i=1}^{n} r_i = \frac{1}{n}*(r_1 + r_2 + ...r_{n-1} + r_n) = \frac{1}{n} * [Q_n*(n-1) + r_n] = Q_n + \frac{1}{n}*(r_n - Q_n) $$    
$$新估计←旧估计+步长[目标−旧估计]$$    
增量公式体现了几个特点:   
1). $n$不是固定的，每一次都会增加1     
2). $\frac{1}{n}$的范围为$[0,1]$,随着尝试次数的增加，会越来越接近于0，使$Q_n$与$Q_{n+1}$逼近,达成收敛。       
3). 可以把这个增量公式看成是动作价值估计$Q$的迭代更新公式(可以类比梯度下降的更新规则)，可以使用 $\alpha \in [0,1)$来替换$\frac{1}{n}$作为步长，当前奖励$r_i$作为目标, $Q_n$最为旧的估计，目标与旧估计的差衡量了更新的方向。     
    
> ** 从增量公式到一般的更新规则，追踪非平稳问题 **:   
$$Q_{n+1}  = Q_n + \alpha*(r_n - Q_n)   , \alpha \in [0,1) $$     
一般更新规则的特点：   
1).此更新规则也称为，指数新近加权平均值 ，因为随着尝试次数的增加，之前获得的奖励$r_i$的权值会越来越小，当$alpha=1$所有权值都落在$r_n$上.   
2).也可以把$\alpha$看作是尝试次数和动作的函数$\alpha_n(a)$，比如 $\alpha=\frac{1}{n}$时，就是样本平均方法.       
3).非平稳问题是指与动作对应的奖励并不服从一个固定的随机分布，而是奖励的随机分布会随着时间会改变，强化学习问题中很多都是非平稳问题。对于非平稳问题，固定步长比较有优势，因为权值分布在最近获得的奖励上以适应变化。  

> ** 目标函数 **: 
多臂赌博机问题的目标函数即使，最大化期望的累积回报。 $$\max E[\sum_{t=1}^T r_{t}] $$   
也等价于，最大化每次选择动作时得到的奖励的期望值，即每次选择具有最大价值的动作。   
但是因为不知道每个动作最对应的价值是多少，所以才产生了选择难题 --- 我需要尝试选择多次之后才能确定哪个动作有最大的价值。   
1). 假如奖励分布的方差为0，那么只需对每个动作尝试一次便知道哪个动作最好，之后便一直选择这个动作即可。     
2). 假如奖励分布的方差不为0，那么需要多次尝试后才能确定出哪个动作最好。    
3). 假如两个奖励分布越接近，那么需要的尝试次数越多，有时可能需要无限尝试才能确定。因此问题难点就集中在，尝试多少次后才能确定某个分布比另一个更好。    
4). 从定性角度来说，好的算法应该具有的品格：尝试的次数较少，并对每个方法都有较充分的探索，并在尝试的过程中能及时丢弃掉明显差的方法，并在最好动作被发现之后便一直利用。    
     

## ** 多臂赌博机的学习方法 **  
> 用于赌博机的学习方法有几种，这些方法的最本质差别在于选择动作的不同，即探索和利用的平衡方式的不同。 
   
> ** 随机选择**:   
随机选择即是说，每次进行选择动作时，对于$n$种动作都不加区别地进行均匀地随机地无休止地进行选择。当然这是最不需要费脑筋的方法，作为所有学习方法效果的一个下限。该方法中，最好动作被选择的概率为$\frac{1}{n}$。  
    
> ** $\epsilon$-greedy 选择方法 **:    
在随机选择方法上加入了一个$1-\epsilon$的概率去选择当前最好的动作，因此不再是不加区分地选择每个动作，而是每次选择动作时都会有一个概率去选择最好的动作，因此最好动作被选择的概率为$ 1-\epsilon + \frac{\epsilon}{n} > \frac{1}{n} $，当$\epsilon$越小，最好地动作被选择到的概率越大。$\epsilon$-greedy 选择方法比随机选择方法更好，因为不再无休止地探索动作，而是在探索过程中利用了最好的动作。但该方法也有自身的缺点，因为探索不足可能找到的最好方法不是最好的那个方法。  
    
> ** 上限置信度选择方法UCB **:     
$$A_t = arg\max_{a} [ Q_t(a) + c\sqrt \frac{\ln(t)}{N_t(a)} ] $$     
对于UCB方法的理解：  
1). 分两部分组成，前者$Q_t(a)$是动作$a$的价值估计,后面$c\sqrt \frac{\ln(t)}{N_t(a)} $是该价值估计的方差或置信区间   
2). 对于同一个动作来说，被选择的次数越多说明价值估计越准确，方差越小.    
3). 对于其他还未被选择的动作来说，更有可能是最好的动作，因此方差越大.    
4). UCB方法比$\epsilon$-greedy方法更好，因为对每个动作都有充分的探索，对最好的动作也有利用，并对两个方法之间明显差的方法不再进行选择，而是对更有可能的方法进行选择。但是该方法的缺点是不易处理非平稳问题。    
    
> ** TS采样方法 **:     
将贝叶斯估计方法引入到动作的选择中，对每个动作的价值都建立一个先验的$beta$分布，每次从先验的$beta$分布中采样一个价值，选择最大价值的动作，根据新得到的奖励来更新$beta$分布的$\alpha$参数和$\beta$参数，从而得到一个后验$beta$分布。如此重复，直到后验分布的方差越来越小，即收敛到期望价值。    

> ** 乐观初始值的影响 **: 在估计动作每一次的价值函数时，初始值可以随机设置，比如为0。如果初始值设置得比动作的实际价值偏大，那么后面无论选择什么动作获得的奖励都比初始值小，从而鼓励去选择其他动作，从而进行充分的探索，可能开始表现会差，但最终会有好的表现。乐观初始值只是一个小技巧，并不适用与非平稳问题。   
    
> ** 学习方法的评价函数 **:  累积遗憾     
累积遗憾来衡量这些方法的好坏，$ regrets=\sum regret_a(a) $。如果这个方法让你感到无比遗憾和后悔时，那么这个方法对你来说就是不好的方法。遗憾$ regret_t(a) = q^* - q^*(a) $,即当选择动作$a$时，动作a的期望值与所有动作的最优期望值之间的差距。 
   

## ** 多臂赌博机实验 **
> 通过多臂赌博机实验来直观感受不同选择方法的效果差异，并帮助更深刻理解这些差异。  

In [ ]:
# 累积回报的边界 对应着 选择算法的边界
# 

**概率**    
> **概率的公理化定义**:   
> **随机变量**：给定一个实验，实验的所有结果构成实验空间$S$，$S$的子集构成的域称为事件，并对这些事件赋予概率。如果对实验的每个结果$\xi$指定一个数$\pmb x(\xi)$，于是建立了一个定义在$S$集合上的函数$\pmb x$，该函数的值域为一个数集，如果该函数满足如下两个条件，则称该函数$\pmb x$为随机变量。    
>> 条件I. 对于每个$x$，集合$\{\pmb x \leq x\}$是一个事件。    
>> 条件II. 事件$\{\pmb x = +\infty\}$和事件$\{\pmb x = -\infty\}$的概率等于0。即，$$P\{\pmb x = +\infty\}=0，P\{\pmb x = -\infty\}=0$$  
>> 总之，随机变量$\pmb x$就是满足上面两个条件的，将每个实验结果$\xi$指定一个数$\pmb x(\xi)$的函数$\pmb x$。  
>> 注：符号$\{\pmb x \leq x\}$表示$S$的一个子集，因此也是一个事件，由满足$\pmb x \leq x$的所有结果$\xi$组成。  
  
> **随机变量的概率分布**：
>> 要点：**从随机变量的定义来看，随机变量是定义在一个给定的实验的所有结果上的，其本质是一个实验结果与数之间的对应函数。符号$\{\pmb x \leq x\}$表示满足$\pmb x \leq x$的所有结果$\xi$组成的一个集合，也就表示为一个事件。**   

>> 分布函数： $$ F(x) = P\{\pmb x \leq x\}$$   $$P\{x_1 < \pmb x \leq x_2\} = F(x_2) - F(x_1)$$    
>> 概率密度函数： $$ f(x) = \frac{\mathrm{d} F(x)}{\mathrm{d} x} = \lim_{\Delta x \rightarrow 0} \frac{\mathrm{d} F(x+\Delta x) - F(x)}{\mathrm{d} x} = \lim_{\Delta x \rightarrow 0} \frac{P\{x < \pmb x \leq x + \Delta x \}}{\mathrm{d} x}, f(x) \geq 0 $$   $$F(x) = \int ^x_{-\infty} f(u) {\rm d}u $$
>> 常见的随机变量类型：   
- 连续型随机变量：均匀分布，正态分布，指数分布，beta分布，伽马分布，卡方分布，...   
- 离散型随机变量：伯努利分布，二项分布，泊松分布，负二项分布，...    
>> 注：   
一般认为很多自然现象都服从正态分布，或者近似为正态分布；beta分布常用在贝叶斯估计中，作为先验分布；伯努利分布是最简单的离散型分布，描述任何只有两个结果的实验或只关心某个事件的出现或不出现的两种结果实验，伯努利随机变量便把这两个结果分别映射为两个不同的值，比如0和1，这两个值描述了事件发生与不发生或者表达成功与失败之类的意义，即$P(x=1)=p,P(x=0)=1-p$，成功概率为$p$，失败概率$1-p$。 如果把n次伯努利实验看成一个实验，那么实验的结果就是某个事件发生或不发生组成的序列，比如4次伯努利实验的一个结果1010，如果存在一个随机变量，把这些0和1组成的序列映射成一个数，这个映射规则为1出现的次数，也表示n次伯努利实验中某个事件发生的次数，那么这样的随机变量就叫做二项式随机变量，其值服从二项式分布。    

> **随机变量函数**：输入为随机变量的函数，输出仍是一个随机变量，概率分布上有什么区别？   
> **随机序列(随机向量)**： 序列的每个维度分量都是一个随机变量  
> **随机过程**：    


** 统计 **：
> ** 均值 **:   
> ** 方差 **:  
> ** 置信区间与置信度 **:   
> ** 估计与预测 **:    

** 信息论 **:
> ** 如何判断两个分布之间的差距有多大 **:    

## 执行一次动作是什么？探索或利用，探索是什么，采样，采样是什么？

## 估计的回报的概率分布的可信度(置信度?)为95%时对应的探索次数是多少？区分两个分布需要的最少探索次数是多少？衡量两个分布的相似性？信息量？信息论？样本复杂度？获得的回报的上下界？

## 当前不同算法的区别是什么？- 对估计的最少探索次数的把握程度？

## 赌博机与强化学习的关系是什么？